# Dimension reduction

## Outline

1. Problem statement
1. Random Projections
1. PCA
1. Truncated SVD
1. NMF
1. Manifold learning
1. TSNE
1. Autoencoders
1. Denoising Autoencoders
1. Sparse Autoencoders

## 1 Problem Statement


Given a set of points $X \in R^{NxD}$, find a map into a lower dimension subspace $f: R^D \rightarrow R^d$ $d  << D$

Depending of the map function there are:  

1. Linear methods: PCA, SVD, NMF, etc..
1. Nonlinear methods

Sometimes, feature selection is considered as dimension reduction.

# 2 Random Projections

<img src="images/proj.png" style="height:300px">

$$ f(X) = X S $$
where $ S \in R^d$

A common example is a gaussian random projection:
1. S is sampled from a normal gaussian distribution
1. each row has unit norm $ ||S_{i,:} || = 1$
1. rows are pairwise orthogonal $ S_{i,:}^T S_{j,:} = 0 $ iff $i \neq j$



Johnson - Lindenstrauss Lemma:   
States, that a set of points in a high-dimensional space can be embedded into a space of much lower dimension in such a way that distances between the points are nearly preserved.  

Given $0 < \epsilon  < 1$, a set of points $X \in R^D$, number $d > \frac {8 \log(m)} {\epsilon^2}$, there exists a linear map $f: R^D \rightarrow R^d$ such that:  
$$ (1-\epsilon) ||u - v||^2 \leq ||f(u) - f(v)||^2 \leq (1+\epsilon) ||u - v||^2$$

There exists a set of points of size $m$ that needs dimension  
$O ( \frac {\log(m)} {\epsilon^2} )$ in order to preserve the distances between all pair of points.

# 3 PCA

Preserve more information while reducing feature dimensions. Here we make assumption that we measure quantity of information by variance. More variable the feature is, more information it contains.

Learning objective:
$$ ||X U - \hat X_k U_k||_F \rightarrow \min_{U}$$, where  $rank(\hat X_k) = k$ and $U_k$ contains first k column vectors of $U$. 


General scheme for PCA:

1. center data $\bar X = X - mean(X)$  
1. build covariance matrix $cov = \bar X \bar X^T$  
1. make low-rank approximation for covariance matrix $X \approx \hat X_k = U_k \Sigma_k U_k^T$ by eigendecomposition
1. sort eigenvalues in descending order by their absolute value
1. select top-k eigenvalues and project data onto corresponding eigenvectors

Learned mapping:
$$f(X) = X U_k$$


Properties:
1. PCA basis vectors creates an uncorrelated orthogonal basis set.  
1. PCA is sensitive to feature scaling
1. variance explained ratio of  j-th principal component = $\frac {|\Sigma_jj|} / {trace(|S|)} $  

<img src="images/pca.png" style="height:300px">

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

SEED = 42
np.random.seed(SEED)

In [2]:
from sklearn.decomposition import PCA

# 4 Truncated SVD

Learning objective:  
$$ || X - \hat X_k||_F \rightarrow min$$
subject to $rank(X_k) = k$.  

which has the exact solution given by SVD decomposition
$$ X \approx \hat X_k = U_k \Sigma_k V^T$$

Learned mapping:  
$ \hat X = X V_k$

Properties:
1. Do not use covariance matrix
1. Do not center data
1. Preferable over PCA for sparse features

<img src="images/svd.png" style="height:300px">

# 5 NMF 

Learnable objective:

$$ || X - WH ||_F \rightarrow \min_{W, H}$$
subject to $X >= 0$, $W >= 0$ and $H >=0 0$.

$$ X \approx \hat X_k = W H $$

Regularized with ElasticNet:  
$ || X - WH ||_F + \gamma *( \alpha(||W||_1 + ||H||_1)  + \frac {1 - \alpha} 2 ( ||W||_2  + ||H||_2)) \rightarrow \min_{W, H} $

Properties:
1. use for tf-idf features
1. more interpretable solution (topic modeling)

<img src="images/nmf.jpg" style="height:400px">

# 6 Manifold learning

Data lies on a low-dimensional non-linear manifold in a high-dimensional space. In other words, features are connected by some non-linear function.


<img src="images/manifold1.png" style="height:300px">

<img src="images/manifold2.png" style="height:300px">

# 7 TSNE

## t-distributed stochastic neighbor embedding


Algorithm:  
1. Compute probability that a sample $x_i$ would peak $x_j$ as its neighbour  
$$ p(j | i) = \frac {1} {Z} \exp (- || x_i - x_j||^2 / 2 \sigma_i^2) $$  
$Z = \sum_{k \neq i} \exp (- || x_i - x_k||^2 / 2 \sigma_i^2) $ - normalization factor

2. probability that $x_i$ and $x_j$ are neighbors:  
$$ p(i,j) = \frac {p(i | j)  + p(j | i)} 2$$  
but $p(i,i) = 0$

3. introduce map $Y \in R^d$, for wich probabolity that $y_i$ and $y_j$ are neighbors:   
$$ q(i, j) = \frac 1 Z (1 + || y_i - y_j||^2 )^{-1}$$
$Z = \sum_{k \neq i} (1 + || y_i - y_k||^2 )^{-1} $ - normalization factor

4. Learning objective = minimize Kullback–Leibler divergence (distance between distributions):
$$ KL(P || Q) = \sum_{i \neq j} p(i,j) \log \frac {p(i,j)} {q(i,j)} \rightarrow \min_{Y} $$


# demo here

# 8 Autoencoders

Autoencoders are feed-forward neural networks, which contains encoder and decoder part. All training methods applicable to feed-forward NN, also can be applied to autoencoders. But unlike feed-forward NN, autoencoders are unsipervised models.

Let $E$ be encoder, $D$ - decoder, $L$  - some loss function.

Than, in the most simple case, the goal is to learn an efficient feature represetation by directing information through a bottleneck and predicting sample itself: 
$$ L(X, D(E(X))) \rightarrow \min_{D, E}$$

<img src="images/autoencoder.png" style="height:300px">

# 9 Denoising autoencoders

Denoising autoencoders try to reconstruct a sample from a noisy one:

$$ \bar X = X + \epsilon $$,
usually we use white noise $\epsilon ~ N(0,1)$

Learning objective:
$$ L(X, D(E(\bar X))) \rightarrow \min_{D, E}$$

# 10 Sparse autoencoders

Sparse autoencoders use another definition of efficiency: good representation should be sparse, so particular sample phenotype will depend only on a small number of features (ideally, 1 feature). It can be achieved by imposing $L_1$ regularization on feature representation.

$$ Z = E(X)$$
$$ L(X, D(Z)) + \alpha ||Z||_1 \rightarrow \min_{D, E} $$

<img src="images/sparse.jpg" style="height:500px">


There are other autoencoder models not discussed here, e.g. variational autoencoders.